# **EECS 3401 Final Project: Understanding Heart Disease Risk**
### Group Members: Deep Patel, Yukta Bhutani, Abdul Wasay Faizan
#### Original Dataset Source: [link](https://www.kaggle.com/datasets/kamilpytlak/personal-key-indicators-of-heart-disease?select=2022)
#### Modified Dataset Source: [link](https://github.com/Deep26053/EECS3401_Heart/blob/main/heart_2022_with_nans.csv)
### Project Description: 
This project aims to analyze a comprehensive dataset related to heart disease risk factors sourced from the Centers for Disease Control and Prevention (CDC). The dataset encompasses survey data from over 400,000 adults collected in 2022, focusing on key indicators associated with heart disease prevalence.

### Attributes for the heart disease dataset:

1. Age: Age of the respondent.
2. Gender: Gender of the respondent.
3. Race: Race of the respondent.
4. HighBloodPressure: Presence of high blood pressure (binary: "Yes" or "No").
5. HighCholesterol: Presence of high cholesterol (binary: "Yes" or "No").
6. SmokingStatus: Smoking status of the respondent.
7. DiabetesStatus: Diabetes status of the respondent.
8. Obesity: Obesity status of the respondent (measured by BMI).
9. PhysicalActivity: Level of physical activity.
10. AlcoholConsumption: Level of alcohol consumption.
11. OtherMedicalConditions: Presence of other medical conditions.
12. FamilyHistory: Family history of heart disease (binary: "Yes" or "No").
13. MedicationUsage: Usage of medications for heart disease or related conditions.
14. StressLevel: Level of stress reported by the respondent.
15. Diet: Dietary habits of the respondent.
16. ExerciseRoutine: Regularity of exercise routines.
17. SleepQuality: Quality of sleep reported by the respondent.
18. SocioeconomicStatus: Socioeconomic status of the respondent.
19. EducationLevel: Level of education attained by the respondent.
20. AccessToHealthcare: Access to healthcare facilities and services.
21. GeographicLocation: Geographic location of the respondent.
22. DateOfSurvey: Date when the survey was conducted.
23. HadHeartAttack: Presence of heart disease (binary: "Yes" or "No").

# 1. Frame the Problem:

### Task: 
Predict the likelihood of heart disease based on demographic and health-related variables using the Heart Disease dataset.

### Key Questions:
1. What are the demographic and health-related variables associated with heart disease risk?
2. How do factors such as age, gender, blood pressure, cholesterol levels, smoking status, diabetes status, obesity, physical activity, and alcohol consumption impact the likelihood of heart disease?
3. What is the distribution of heart disease cases among different demographic groups?
4. Can we identify patterns or correlations between variables and heart disease prevalence?
5. Which machine learning algorithm provides the most accurate predictions for heart disease likelihood?

- **Supervised Learning:** Since the dataset includes labelled data (e.g., presence or absence of heart disease), it is a supervised learning problem.
- **Classification Task:** The objective is to predict whether an individual is at risk of heart disease (binary classification).
- **Batch Learning:** The dataset represents a finite set of data collected at a specific time, making batch learning suitable for model training. There is no continuous flow of data coming into the system, and there is no need to adjust to changing data rapidly.
This framing of the problem provides clarity on the type of learning approach, the nature of the prediction task, and the appropriate learning technique to use for developing the heart disease prediction model.


# Look at the Big Picture:
The model's output (prediction of an individual's likelihood of heart disease) will be used as one of many signals in a broader healthcare system. This downstream system will aid healthcare professionals in making informed decisions regarding patient care, such as recommending preventive measures, lifestyle modifications, or medical interventions.

## Key Analyses:
- **Distribution of Heart Disease Risk Factors:**
Explored the prevalence and distribution of heart disease risk factors including blood pressure, cholesterol levels, smoking status, diabetes status, obesity, physical activity levels, and alcohol consumption to identify common patterns and variations.
- **Impact of Demographic Factors on Heart Disease Risk:**
Investigated the relationship between demographic factors (age, gender, ethnicity) and heart disease prevalence to identify any significant associations or disparities.
- **Correlation between Risk Factors and Heart Disease:**
Identified correlations between variables such as blood pressure, cholesterol levels, smoking status, and heart disease to understand their impact on heart disease risk.
- **Feature Importance Analysis:**
This analysis can provide insights into which demographic and health-related factors have the greatest impact on heart disease risk.

### Impact: 
By accurately predicting the likelihood of heart disease, the model can contribute to reducing the burden of cardiovascular disease, improving patient outcomes, and reducing healthcare costs associated with preventable heart-related complications.


In [2]:
# Import the necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer

# 2. Load Dataset:
Open the dataset using Pandas and load it into a DataFrame, which is the object Pandas uses to store tables of data.
Pandas uses two objects for storing datasets: the DataFrame and the Series.
Series is used for datasets with only one column, and DataFrame is used for datasets of more than one column.

In [3]:
heart_data = pd.read_csv('heart_2022_with_nans.csv') # Read dataset from the CSV file into a DataFrame

In [4]:
heart_data.head() # Use head() to look at the first 5 rows

,version https://git-lfs.github.com/spec/v1
0,oid sha256:8437ca7cc35bd5e515ae8242982d4210472...
1,size 139870877


In [5]:
heart_data.describe() # Use describe() method to see a summary of the numerical attributes

,version https://git-lfs.github.com/spec/v1
count,2
unique,2
top,oid sha256:8437ca7cc35bd5e515ae8242982d4210472...
freq,1


In [6]:
heart_data.info() # Use info() to get a quick description of the data 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 1 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   version https://git-lfs.github.com/spec/v1  2 non-null      object
dtypes: object(1)
memory usage: 148.0+ bytes


In [7]:
heart_data.shape #Determine the shape of the dataset

(2, 1)

# 2. Cleaning Dataset

In [8]:
missing_values = heart_data.isnull().sum() #Check for missing values

In [9]:
# Remove columns with more than 50% missing values
heart_data.drop(columns = missing_values[missing_values > 0.5].index)

,version https://git-lfs.github.com/spec/v1
0,oid sha256:8437ca7cc35bd5e515ae8242982d4210472...
1,size 139870877


In [10]:
def handle_non_numerical_data(df):
    columns = df.columns.values
    for column in columns:
        text_digit_vals = {}
        def convert_to_int(val):
            return text_digit_vals[val]

        if df[column].dtype != np.int64 and df[column].dtype != np.float64:
            column_contents = df[column].values.tolist()
            unique_elements = set(column_contents)
            x = 0
            for unique in unique_elements:
                if unique not in text_digit_vals:
                    text_digit_vals[unique] = x
                    x+=1

            df[column] = list(map(convert_to_int, df[column]))

    return df

# Apply the function to the dataset
heart_data = handle_non_numerical_data(heart_data)

In [11]:
# Convert missing values to median
df = heart_data
fill_values = {col: df[col].median() for col in df.columns if df[col].isnull().any()}
df.fillna(fill_values, inplace=True)



In [12]:
heart_data.isnull().sum()

version https://git-lfs.github.com/spec/v1    0
dtype: int64

In [ ]:
heart_data.drop(columns=['RemovedTeeth', 'HadDepressiveDisorder', 'HIVTesting', 'FluVaxLast12', 'PneumoVaxEver', 'TetanusLast10Tdap', 'PneumoVaxEver', 'CovidPos', 'PneumoVaxEver', 'HighRiskLastYear', 'ChestScan', 'DifficultyErrands', 'DifficultyDressingBathing', 'DifficultyWalking', 'DifficultyConcentrating', 'BlindOrVisionDifficulty'], inplace=True)

# **3. Explore and visualize the data to gain insights**

**3.1 Distribution of Heart Disease Risk Factors:**

In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


# Calculate the percentage of people with heart disease by different categories
categories = ['HadDiabetes', 'SmokerStatus', 'ECigaretteUsage', 'BMI', 'AlcoholDrinkers', 'PhysicalActivities']
percentages = []
for category in categories:
    total = len(heart_data[category])
    positive = len(heart_data[(heart_data[category] == 1) & (heart_data['HadHeartAttack'] == 1)])
    percent = round(positive / total * 100, 2)
    percentages.append(percent)

# Create a pie chart
labels = ['Diabetes', 'Smoker', 'E-Cigarette', 'BMI', 'Alcohol', 'Physical Activity']
plt.pie(percentages, labels=labels, autopct='%1.1f%%')
plt.axis('equal')
plt.title('Percentage of People with Heart Disease by Different Categories')
plt.show()

KeyError: 'HadDiabetes'

In [21]:
print(heart_data.columns)

Index(['version https://git-lfs.github.com/spec/v1'], dtype='object')
